[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/juansensio/blog/blob/master/090_dlops_onnx/090_dlops_onnx.ipynb)

# DLOps - ONNX

Una vez hemos entrenado varios modelos, los hemos comparado y hemos decidido usar uno, tenemos que ponerlo en producción para que sea accesible. Para ello tenemos diferentes alternativas, en función de la aplicación (desde desplegar un modelo en dispositivos móviles o IoT hasta en servidores en la nube accesibles a través de una API). En esta serie de posts asumiermos que nuestro modelo será ejecutado en un servidor en la nube, lo cual es lo más común ya que de esta manera podemos controlar los recursos computacionales disponibles para su ejecución, monitorizarlo, desplegar nuevas versiones fácilmente, etc. De nuevo, para ello tenemos diferentes opciones. En nuestro caso, que hemos entrenado los modelos usando Pytorch y Pytorch Lightning, podríamos usar cualquier *framework* en Python que nos permita servir las predicciones a través de internet, como por ejemplo [Flask](https://flask.palletsprojects.com/en/2.0.x/) o [FastAPI](https://fastapi.tiangolo.com/). El principal problema de esta opción es que tendremos que cargar todas las librerías (y sus dependencias) en nuestra API, lo cual resultará en una carga muy pesada. Recientemente, Pytorch incluye una solución dedicada para este caso de uso, [Torchserve](https://pytorch.org/serve/) que si bien nos ofrece una solución optimizada para servir modelos en producción, está limitada al uso de modelos en Pytorch.

Es en este punto en el que entra [ONNX](https://onnx.ai/), un estándar abierto para la representación de redes neuronales que permite la interoperabilidad entre librerías y ofrece una solución optimizada para servir modelos en producción (tanto en la nube como on dispositvos móviles). De esta manera podemos desacoplar el entrenamiento de los modelos de su puesta en producción, utilizando en cada caso las herramientas preferidas para su desarrollo.

## Exportar un modelo a ONNX

Vamos a ver como podemos exportar un modelo entrenado a ONNX. En primer lugar, cargaremos el *checkpoint* deseado.

In [3]:
from src import *

module = MNISTModule.load_from_checkpoint('checkpoints/006-val_loss=0.14715-epoch=7.ckpt')
module.mlp

Sequential(
  (0): Linear(in_features=784, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=1, bias=True)
)

Es una buena práctica evaluar nuestro modelo antes y después de exportarlo para asegurarnos de que todo funciona correctamenete.

In [4]:
import torch 

dm = MNISTDataModule(**module.hparams['datamodule'])
dm.setup()
module.eval()
with torch.no_grad():
    preds, labels = torch.tensor([]), torch.tensor([])
    for imgs, _labels in dm.val_dataloader():
        outputs = module.predict(imgs) > 0.5
        preds = torch.cat([preds, outputs.cpu().long()])
        labels = torch.cat([labels, _labels])

acc = (preds == labels).float().mean()
acc.item()

0.949999988079071

In [6]:
imgs.shape

torch.Size([5, 28, 28])

Pytorch Lightning nos permite exportar un modelo a ONNX de manera muy sencilla con la siguiente línea.

In [7]:
input_sample = torch.randn((1,28,28))
module.to_onnx('models/006.onnx', input_sample, export_params=True)

/home/juan/miniconda3/lib/python3.8/site-packages/torch/onnx/symbolic_opset9.py:688: UserWarning: ONNX export squeeze with negative axis -1 might cause the onnx model to be incorrect. Negative axis is not supported in ONNX. Axis is converted to 1 based on input shape at export time. Passing an tensor of different rank in execution will be incorrect.
  warnings.warn("ONNX export squeeze with negative axis " + str(squeeze_dim) +
/home/juan/miniconda3/lib/python3.8/site-packages/torch/onnx/symbolic_opset9.py:714: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn("This model contains a squeeze operation on dimension " + str(squeeze_dim) + ". If the model is " +


In [9]:
import onnxruntime

ModuleNotFoundError: No module named 'onnxruntime'